# DMA to streamed interfaces example

Overlay consists of two DMAs and an AXI Stream FIFO (input and output AXI stream interfaces). The FIFO represents an accelerator.

One DMA with read channel enabled is connected from DDR to IP input stream. (Reads from DDR, sends to AXI stream)
The other DMA has a write channel enabled and is connected to IP output stream to DDR. (received from AXI stream, writes to DDR)

There are other IP in the design which will be ignored for now. 

![Stream Example](data/dma_stream_example.png)

In [1]:
from pynq import Overlay
ol = Overlay("pynq_tutorial.bit")
ol.download()

Check addresses of IP in the overlay

In [2]:
ol.ip_dict

{'SEG_axi_dma_from_pl_to_ps_Reg': [1078001664, 65536, None],
 'SEG_axi_dma_from_ps_to_pl_Reg': [1077936128, 65536, None],
 'SEG_btns_gpio_Reg': [1092681728, 65536, None],
 'SEG_mb_bram_ctrl_1_Mem0': [1073741824, 65536, None],
 'SEG_mb_bram_ctrl_2_Mem0': [1107296256, 65536, None],
 'SEG_rgbleds_gpio_Reg': [1092747264, 65536, None],
 'SEG_swsleds_gpio_Reg': [1092616192, 65536, None],
 'SEG_system_interrupts_Reg': [1098907648, 65536, None]}

Create DMA instances

In [3]:
from pynq.drivers import DMA

# dma_read reads from DDR and sends to Stream
dma_read_addr = ol.ip_dict["SEG_axi_dma_from_ps_to_pl_Reg"][0]
dma_read = DMA(dma_read_addr, 0)  # 'DMA_TO_DEV'

dma_write_addr = ol.ip_dict["SEG_axi_dma_from_pl_to_ps_Reg"][0]
dma_write = DMA(dma_write_addr, 1) # 'DMA_FROM_DEV'

## Debug DMA

Create some debug functions to print control and status info from DMAs

In [4]:
from pynq import MMIO
dma_read_s = MMIO(dma_read_addr, 128)
dma_write_s = MMIO(dma_write_addr, 128)

def print_dma_status():

    print("Read from Memory, Write to FIFO")

    print("MM 2 Stream        Ctrl   : " + format(dma_read_s.read(0x0), '02x'))
    print("Binary                    : " + format(dma_read_s.read(0x0), '0b'))
    print("MM 2 Stream        Status : " + format(dma_read_s.read(0x4), '02x'))
    print("Binary                    : " + format(dma_read_s.read(0x4), '0b'))
    
    print("\nRead from FIFO, Write to Memory")
    
    print("Stream to MM       Ctrl   : " + format(dma_write_s.read(0x30), '02x'))
    print("Binary                    : " + format(dma_write_s.read(0x30), '0b'))
    print("Stream to MM       Status : " + format(dma_write_s.read(0x34), '02x'))
    print("Binary                    : " + format(dma_write_s.read(0x34), '0b'))

def dma_reset_irq():
    control = dma_read_s.read(0x4)
    control = control | 0x1000
    dma_read_s.write(0x4, control)
    
    control = dma_write_s.read(0x34)
    control = control | 0x1000
    dma_write_s.write(0x34, control)

In [5]:
print_dma_status()
# Control 10002
# (1) : Alwyas 1
# (16): IRQ threshold 1
# Status 01
# (0) : 1 = Halted

Read from Memory, Write to FIFO
MM 2 Stream        Ctrl   : 10002
Binary                    : 10000000000000010
MM 2 Stream        Status : 01
Binary                    : 1

Read from FIFO, Write to Memory
Stream to MM       Ctrl   : 10002
Binary                    : 10000000000000010
Stream to MM       Status : 01
Binary                    : 1


# Read DMA
Read from memory, write to FIFO

## Create the DMA buffer 

In [6]:
dma_read.create_buf(1024)

## Get the buffer pointer



In [7]:
read_buffer = dma_read.get_buf(32)
print(read_buffer)

<cdata 'unsigned int *' 0x36fab000>


## Write some test data to the buffer
This data will be transferred by the DMA to the FIFO. 

In [8]:
transfer_size = 10
for i in range(transfer_size):
    read_buffer[i] = i+0xcafe0000;

## Print data
Check the contents of the buffer

In [9]:
for i in range(transfer_size):
    print(format(read_buffer[i], '02x'))

cafe0000
cafe0001
cafe0002
cafe0003
cafe0004
cafe0005
cafe0006
cafe0007
cafe0008
cafe0009


## Carry out DMA transfer from buffer in DDR to FIFO

In [10]:
#0 DMA_TO_DEV
dma_read.transfer(transfer_size*4, 0)

In [11]:
print_dma_status()
# Control : 10003
# (0) Run/Stop :1 = Run start DMA ops. Halted bit -> 0
# (1) : Always 1
# (16) IRQ threshold
# Status 
# (0) = 0 ; Halted
# (1) = 1 ; idle
# (12) Interrupt on complete

Read from Memory, Write to FIFO
MM 2 Stream        Ctrl   : 10003
Binary                    : 10000000000000011
MM 2 Stream        Status : 1002
Binary                    : 1000000000010

Read from FIFO, Write to Memory
Stream to MM       Ctrl   : 10002
Binary                    : 10000000000000010
Stream to MM       Status : 01
Binary                    : 1


# Write DMA
Read from FIFO stream, write to MM memory


## Create a buffer

In [12]:
dma_write.create_buf(1024)

## Get pointer to buffer

In [13]:
write_buffer = dma_write.get_buf(32)

## Check buffer before DMA transfer

In [14]:
for i in range(transfer_size):
    print(format(write_buffer[i], '02x'))

00
00
00
00
00
00
00
00
00
00


## Carry out DMA transfer from FIFO to buffer in DDR

In [15]:
#1 DEV_TO_DMA
dma_write.transfer(transfer_size*4, 1)

In [16]:
print_dma_status()

Read from Memory, Write to FIFO
MM 2 Stream        Ctrl   : 10003
Binary                    : 10000000000000011
MM 2 Stream        Status : 1002
Binary                    : 1000000000010

Read from FIFO, Write to Memory
Stream to MM       Ctrl   : 10003
Binary                    : 10000000000000011
Stream to MM       Status : 1002
Binary                    : 1000000000010


## Check contents of buffer in DDR to confirm transfer was successful

In [17]:
for i in range(transfer_size):
    print(format(write_buffer[i], '02x'))

cafe0000
cafe0001
cafe0002
cafe0003
cafe0004
cafe0005
cafe0006
cafe0007
cafe0008
cafe0009
